# Necessary Files installation and import

In [ ]:
import os 
os.chdir("/content/drive/My Drive/Colab Notebooks")
import time

In [ ]:
!pip install lexical_diversity

     |████████████████████████████████| 122kB 3.4MB/s 


In [ ]:
!pip install textstat

     |████████████████████████████████| 102kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 11.2MB/s 


In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import tokenize
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from lexical_diversity import lex_div as ld
from nltk.util import ngrams
import gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import os
import codecs
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
from collections import OrderedDict
# import nlp
import spacy
from textstat.textstat import textstatistics, legacy_round
import en_core_web_sm
from sklearn.preprocessing import StandardScaler
nlp = en_core_web_sm.load()
nlp.max_length = 3050000

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

# Feature extraction part


In [ ]:
#  In order to extract features from our dataset, we splitted the csv with 996 records into 6 smaller parts, 
#  like master_part1-0-100.csv , master_part2-100-300.csv , master_part3-300-500.csv, 
#  master_part4-500-700.csv, master_part5-700-950.csv, master_part6-950-996.csv , with the resources available, this spliting helped for our requirement.
#  The results shown here are only for 300-500 data points.

start_time = time.time()

#  This function helps us to count how many he and she pronouns are present in the dataset, first it removes irrelevant bigrams 
# and then it compares key[0] of bigram to count he and she store them in arrays.
def gender_roles(data):
    he_count = []
    she_count = []
    pronouns = ['he', 'she']
    for idx, book in enumerate(data.bigram):
        c1 = 0
        c2 = 0
        for key, val in list(book.items()):
            if key[0] not in pronouns:
                del book[key]
            if key[0] == pronouns[0]:
                c1 += 1

            elif key[0] == pronouns[1]:
                c2 += 1
        he_count.insert(idx, c1)
        she_count.insert(idx, c2)
    data['He Pronoun'] = he_count
    data['She Pronoun'] = she_count
    return data

# Reference from: https://www.geeksforgeeks.org/readability-index-pythonnlp/
#  This function makes use of nlp object of spaCy to return documents, from which further sentences are returned.
def sentence_break(sentence):
    # nlp = spacy.load('en')
    doc = nlp(sentence)
    return doc.sents

# Function is used to count total words present in all the sentences of a data point
def word_count(sentence):
    sentences = sentence_break(sentence)
    words = 0
    for sen in sentences:
        words += len([token for token in sen])
    return words

# Function is used to count total sentences present in the data point
def sentence_count(sentence):
    sentences = sentence_break(sentence)
    count = 0
    for sen in sentences:
        count += 1
    return count

#  This function is calculating average sentence of a data point i.e. by dividing total number of words by total number of sentences.
def avg_sentence_length(sentence):
    words = word_count(sentence)
    sentences = sentence_count(sentence)
    if sentences == 0:
      return 0
    else:
      average_sentence_length = float(words / sentences)
      return average_sentence_length

# It makes use of API textstat to return sum of syllables for each word in a sentence for all the sentences present in the data point.
def syllables_count(sentence):
    return textstatistics().syllable_count(sentence)

#  This function is calculating average syllables per word for a data point i.e. by dividing total number of syllables by total number of words.
def avg_syllables_per_word(sentence):
    syllable = syllables_count(sentence)
    words = word_count(sentence)
    if words == 0:
      return 0
    else:
      average_syllables_per_word = float(syllable) / float(words)
      return legacy_round(average_syllables_per_word, 1)

# Implementing Flesch Formula:
# Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
# where
# ASL is average sentence length (number of words divided by number of sentences)
# and ASW is average word length in syllables (number of syllables divided by number of words)

def flesch_reading_score(sentence):
    flesch_read_score = 206.835 - float(1.015 * avg_sentence_length(sentence)) - float(
        84.6 * avg_syllables_per_word(sentence))
    return legacy_round(flesch_read_score, 2)

In [ ]:
    # Read the csv file into pandas dataframe for 3rd part we have shown here.
    data = pd.read_csv("master_part3_300-500.csv", encoding="ISO-8859-1")

    # Replace book id with the actual name present in html so as to iterate through html files
    data['book_id'] = data['book_id'].str.replace('.epub', '-content.html', case=False)
    book_id_array = data['book_id'].to_numpy()
    # print(book_id_array[0])

    genre = ['guten_genre']

    # Convert each genre of data dataframe into numbers
    for x in genre:
        le = LabelEncoder()
        le.fit(list(data[x].values))
        data[x] = le.transform(list(data[x]))

    print('******************Dataframe after converting Genre as numbers**********************')
    print(data)

    #  Declaration of all the empty arrays 
    wordArray = []
    contentList = []
    index = 0
    tokensList = []
    sentenceList = []
    ttrList = []
    topicList = []
    bigramArray = []
    personCountList = []
    flesch_reading_score_list = []

    # Iterate through each .html files and read the contents and append to our exisitng dataframe
    while index < len(book_id_array):

        # Open the HTML file and read the contents
        with open('/content/drive/My Drive/Colab Notebooks/Gutenberg_19th_century_English_Fiction/' + book_id_array[index], 'r', encoding='utf-8') as HTMLfile:
          content = HTMLfile.read()
          r = re.compile('<.*?>')
          content_string = re.sub(r, '', content)

          # Use this for finding the length of sentence
          sent_tokens = tokenize.sent_tokenize(content_string)
          print('*********************Sentence Tokens*************************************')
          print('Book: ', book_id_array[index], 'Sentence Length: ', len(sent_tokens))

          prunc_removed_str = re.sub('[^A-Za-z0-9]+', ' ', content_string)
          print('Document ', index, 'is parsing..')

          # Tokenize words
          tokens = ld.tokenize(prunc_removed_str)
          # TTR is type token ratio which we are using for calculating lexical diversity in the data point and appending it into a array.
          ttr = ld.hdd(tokens)
          ttrList.insert(index, ttr)

          # Generating Entities for all the sentences and choosing Person Entity Label to extract number of characters feature.
          # Reference from: https://www.geeksforgeeks.org/python-named-entity-recognition-ner-using-spacy/
          labels = dict([(str(x), x.label_) for x in nlp(str(content_string)).ents])
          person_count = [name for name, ent in labels.items() if ent == 'PERSON']
          
          # Tokenization, changing to lower case and stop word removal.
          tokens = word_tokenize(content_string)
          words = [word for word in tokens if word.isalpha()]
          filtered_words = [w for w in words if not w in stopwords.words('english')]

          bigram_topic = list(ngrams(filtered_words, 2))
          bigram = Counter(list(ngrams(words, 2)))
          wordArray.insert(index, words)
          bigramArray.insert(index, bigram)
          contentList.insert(index, prunc_removed_str)
          sentenceList.insert(index, len(sent_tokens))
          personCountList.insert(index, len(person_count))

          index = index + 1
      
    # After removing punctuations, storing the content in dataframe
    data['tokens'] = wordArray
    data['bigram'] = bigramArray
    data['content'] = contentList

    # Pass the contents to SentimentIntensityAnalyser
    sid = SentimentIntensityAnalyzer()
    data['scores'] = data['content'].apply(lambda content: sid.polarity_scores(content))

    # Compound is the value which determines if the doc is positive or negative ranges between -1 to +1, anything above 0 is positive
    data['compound'] = data['scores'].apply(lambda score_dict: score_dict['compound'])

    # Sentiment based on compound score
    data['sentiment'] = data['compound'].apply(lambda c: 'pos' if c >= 0 else 'neg')
    
    # Calling Gender Roles function
    gender_roles(data)

    #  For each sentence in data's column content flesch reading score function is called and the score is stored in array later.
    for idx, sentence in enumerate(data['content']):
        flesch_read_score = flesch_reading_score(sentence)
        flesch_reading_score_list.insert(idx, flesch_read_score)

    data['Ease Of Readability'] = flesch_reading_score_list

    # Printing data frame
    print(data)

    print('*************************Scores List*****************************')
    # Consists of List of dict with the score having pos neg and neutral
    scoresList = list(data['scores'])
    print(scoresList)

    # Seperate pos, neg and neutral and merge the array for vectorizing
    posArray = [dic['pos'] for dic in scoresList]
    negArray = [dic['neg'] for dic in scoresList]
    neuArray = [dic['neu'] for dic in scoresList]
    featureVec = np.vstack((posArray, neuArray, negArray)).T
    print('Feature Vector', featureVec)

    # Final Dataframe with Extracted features
    dfFeature = pd.DataFrame(data=featureVec, columns=['postive', 'neutral', 'negative'])
    dfFeature['sentence_length'] = sentenceList
    dfFeature['TTR'] = ttrList
    dfFeature['person_count'] = personCountList
    dfFeature['she_pronoun'] = data['She Pronoun']
    dfFeature['he_pronoun'] = data['He Pronoun']
    dfFeature['ease_of_readability'] = data['Ease Of Readability']
    dfFeature['class'] = data['guten_genre']

    # Converting Dataframe into csv
    dfFeature.to_csv(r'export_dataframe.csv', index=False, header=True)

    print('**************************Feature Df*****************************')
    print(dfFeature)

end_time = time.time()

******************Dataframe after converting Genre as numbers**********************
                                             Book_Name  ...             Author_Name
0    The Bradys Beyond Their Depth- The Great Swamp...  ...    Bird| Frederic Mayer
1                                     Bird of Paradise  ...  Seltzer| Charles Alden
2                                     The Giant's Robe  ...         Sabin| Edwin L.
3                                    The Seven Secrets  ...              Anstey| F.
4                              Five O'Clock Tea: Farce  ...       Queux| William Le
..                                                 ...  ...                     ...
194                                 The Mynns' Mystery  ...   Fenn| George Manville
195                      The Man with the Double Heart  ...            Hine| Muriel
196                                     One of My Sons  ...   Green| Anna Katharine
197                      The Ivory Gate- a new edition  ...          Besant|

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:155: DeprecationWarning: generator 'ngrams' raised StopIteration
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:156: DeprecationWarning: generator 'ngrams' raised StopIteration


*********************Sentence Tokens*************************************
Book:  pg34166-content.html Sentence Length:  5858
Document  167 is parsing..
*********************Sentence Tokens*************************************
Book:  pg34168-content.html Sentence Length:  5351
Document  168 is parsing..
*********************Sentence Tokens*************************************
Book:  pg34202-content.html Sentence Length:  3984
Document  169 is parsing..
*********************Sentence Tokens*************************************
Book:  pg34244-content.html Sentence Length:  7928
Document  170 is parsing..
*********************Sentence Tokens*************************************
Book:  pg34246-content.html Sentence Length:  11453
Document  171 is parsing..
*********************Sentence Tokens*************************************
Book:  pg34248-content.html Sentence Length:  9537
Document  172 is parsing..
*********************Sentence Tokens*************************************
Book:  pg3427

In [ ]:
print((end_time-start_time) / 60)

252.70055793921154
